In [1]:
import torch
import torch.nn as nn
import numpy as np
import scipy.io
import matplotlib.pyplot as plt

In [2]:
#nu = 0.00000016 # From very high Reynolds number
nu = 0.01
def create_pde(net, x, y, t):
    res = net(torch.hstack((x, y, t)))
    psi, p = res[:, 0:1], res[:, 1:2]

    u = torch.autograd.grad(psi, y, grad_outputs=torch.ones_like(psi), create_graph=True)[0]  
    v = -1. * torch.autograd.grad(psi, x, grad_outputs=torch.ones_like(psi), create_graph=True)[0]

    u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_xx = torch.autograd.grad(u_x, x, grad_outputs=torch.ones_like(u_x), create_graph=True)[0]
    u_y = torch.autograd.grad(u, y, grad_outputs=torch.ones_like(u), create_graph=True)[0]
    u_yy = torch.autograd.grad(u_y, y, grad_outputs=torch.ones_like(u_y), create_graph=True)[0]
    u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u), create_graph=True)[0]

    v_x = torch.autograd.grad(v, x, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_xx = torch.autograd.grad(v_x, x, grad_outputs=torch.ones_like(v_x), create_graph=True)[0]
    v_y = torch.autograd.grad(v, y, grad_outputs=torch.ones_like(v), create_graph=True)[0]
    v_yy = torch.autograd.grad(v_y, y, grad_outputs=torch.ones_like(v_y), create_graph=True)[0]
    v_t = torch.autograd.grad(v, t, grad_outputs=torch.ones_like(v), create_graph=True)[0]

    p_x = torch.autograd.grad(p, x, grad_outputs=torch.ones_like(p), create_graph=True)[0]
    p_y = torch.autograd.grad(p, y, grad_outputs=torch.ones_like(p), create_graph=True)[0]

    f = u_t + u * u_x + v * u_y + p_x - nu * (u_xx + u_yy)
    g = v_t + u * v_x + v * v_y + p_y - nu * (v_xx + v_yy)

    return u, v, p, f, g

# Initialize the network
class ResNetBlock(nn.Module):
    def __init__(self, in_features, out_features):
        super(ResNetBlock, self).__init__()
        self.linear = nn.Linear(in_features, out_features)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        return self.activation(self.linear(x)) + x

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.input_layer = nn.Linear(3, 100)
        self.res_block1 = ResNetBlock(100, 100)
        self.res_block2 = ResNetBlock(100, 100)
        self.res_block3 = ResNetBlock(100, 100)
        self.res_block4 = ResNetBlock(100, 100)
        self.res_block5 = ResNetBlock(100, 100)
        self.res_block6 = ResNetBlock(100, 100)
        self.res_block7 = ResNetBlock(100, 100)
        self.res_block8 = ResNetBlock(100, 100)
        self.res_block9 = ResNetBlock(100, 100)
        self.res_block10 = ResNetBlock(100, 100)
        self.res_block11 = ResNetBlock(100, 100)
        self.res_block12 = ResNetBlock(100, 100)
        self.res_block13 = ResNetBlock(100, 100)
        self.res_block14 = ResNetBlock(100, 100)
        self.res_block15 = ResNetBlock(100, 100)
        self.res_block16 = ResNetBlock(100, 100)
        self.res_block17 = ResNetBlock(100, 100)
        self.res_block18 = ResNetBlock(100, 100)
        self.res_block19 = ResNetBlock(100, 100)
        self.res_block20 = ResNetBlock(100, 100)
        self.output_layer = nn.Linear(100, 3)
        self.activation = nn.Tanh()
    
    def forward(self, x):
        x = self.activation(self.input_layer(x))
        x = self.res_block1(x)
        x = self.res_block2(x)
        x = self.res_block3(x)
        x = self.res_block4(x)
        x = self.res_block5(x)
        x = self.res_block6(x)
        x = self.res_block7(x)
        x = self.res_block8(x)
        x = self.res_block9(x)
        x = self.res_block10(x)
        x = self.res_block11(x)
        x = self.res_block12(x)
        x = self.res_block13(x)
        x = self.res_block14(x)
        x = self.res_block15(x)
        x = self.res_block16(x)
        x = self.res_block17(x)
        x = self.res_block18(x)
        x = self.res_block19(x)
        x = self.res_block20(x)
        x = self.output_layer(x)
        return x

net = ResNet()
state = torch.load('./models/20x100lyrs_NACA0012_extrapolate(0-140).pt')
net.load_state_dict(state)
print("Model loaded.")

Model loaded.


In [14]:
# Load test data
test_file = 'Cleaned_APS360_0012_dt0001_2.mat'
test_data = scipy.io.loadmat(f"./Data/{test_file}")

##### Bounds for testing ######
start_time = 0
end_time = 140

# Change to linspace/add slicing here
U_test = test_data['U_star'][...,start_time:end_time]  # 5000 x 2 x 200
P_test = test_data['p_star'][...,start_time:end_time]  # 5000 x 200
T_test = test_data['t'][start_time:end_time]  # 200 x 1
X_test = test_data['X_star']  # 5000 x 2

print(U_test.shape, P_test.shape, T_test.shape, X_test.shape)

N = X_test.shape[0]
T = T_test.shape[0]

# Null vector to test against f and g, and error func
null_vector = torch.zeros((N, 1))
mse = nn.MSELoss()

u_loss_sum, v_loss_sum, p_loss_sum, f_loss_sum, g_loss_sum = [0]*5

for time in range(T):
    # Tidy up data
    UU = U_test[:, 0, time]  # N x T
    VV = U_test[:, 1, time]  # N x T
    PP = P_test[...,time]  # N x T

    TT = np.tile(T_test[time], (1, N))  # N x T

    x_test = X_test[:, 0:1].flatten()[:, None]  # NT x 1
    y_test = X_test[:, 1:2].flatten()[:, None]  # NT x 1
    t_test = TT.flatten()[:, None]  # NT x 1

    u_test = UU.flatten()[:, None]  # N x 1
    v_test = VV.flatten()[:, None]  # N x 1
    p_test = PP.flatten()[:, None]  # N x 1

    # Convert test data to tensors
    x = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
    y = torch.tensor(y_test, dtype=torch.float32, requires_grad=True)
    t = torch.tensor(t_test, dtype=torch.float32, requires_grad=True)
    u = torch.tensor(u_test, dtype=torch.float32)
    v = torch.tensor(v_test, dtype=torch.float32)
    p = torch.tensor(p_test, dtype=torch.float32)
    
    # Get outputs
    u_prediction, v_prediction, p_prediction, f_prediction, g_prediction = create_pde(net, x, y, t)
    u_loss = mse(u_prediction, u)
    v_loss = mse(v_prediction, v)
    p_loss = mse(p_prediction, p)
    f_loss = mse(f_prediction, null_vector)
    g_loss = mse(g_prediction, null_vector)

    u_loss_sum += u_loss.item()
    v_loss_sum += v_loss.item()
    p_loss_sum += p_loss.item()
    f_loss_sum += f_loss.item()
    g_loss_sum += g_loss.item()

    loss = u_loss + v_loss + p_loss + f_loss + g_loss

    print(f"Timestep {time} loss: {loss.item()}")

    # Reverse the manipulations done on input data
    #u_prediction_rs = np.reshape(u_prediction.detach().numpy(), UU.shape)
    #v_prediction_rs = np.reshape(v_prediction.detach().numpy(), VV.shape)
    #p_prediction_rs = np.reshape(p_prediction.detach().numpy(), PP.shape)

# Save to .mat for display
#t_prediction_rs = T_test[start_time:end_time]
#U_prediction_rs = np.stack([u_prediction_rs, v_prediction_rs], 1)

#print(U_prediction_rs.shape, p_prediction_rs.shape)
#scipy.io.savemat(f'./Results/run_{test_file}', {'U_star': U_prediction_rs, 'p_star': p_prediction_rs, 'X_star': X_test, 't': t_prediction_rs})

print(u_loss_sum/T, v_loss_sum/T, p_loss_sum/T)
print(f_loss_sum/T, g_loss_sum/T)

(14449, 2, 140) (14449, 140) (140, 1) (14449, 2)
Timestep 0 loss: 18821794496512.0
Timestep 1 loss: 359579090944.0
Timestep 2 loss: 5120379584512.0
Timestep 3 loss: 135696695296.0
Timestep 4 loss: 34304268288.0
Timestep 5 loss: 275976832.0
Timestep 6 loss: 2609897472.0
Timestep 7 loss: 424550662144.0
Timestep 8 loss: 34138253312.0
Timestep 9 loss: 43623112704.0
Timestep 10 loss: 54683716.0
Timestep 11 loss: 8412302848.0
Timestep 12 loss: 243248201728.0
Timestep 13 loss: 199113637888.0
Timestep 14 loss: 932379584.0
Timestep 15 loss: 3688470151168.0
Timestep 16 loss: 16273714970624.0
Timestep 17 loss: 12295359954944.0
Timestep 18 loss: 80639229952.0
Timestep 19 loss: 7050674831360.0
Timestep 20 loss: 1537556873216.0
Timestep 21 loss: 2302650089472.0
Timestep 22 loss: 4706911232.0
Timestep 23 loss: 232109703168.0
Timestep 24 loss: 4630586064896.0
Timestep 25 loss: 475844050944.0
Timestep 26 loss: 12214240256.0
Timestep 27 loss: 697192939520.0
Timestep 28 loss: 1146166837248.0
Timestep 29 

In [15]:
# Load test data
test_file = 'Cleaned_APS360_0012_dt0001_2.mat'
test_data = scipy.io.loadmat(f"./Data/{test_file}")

##### Bounds for testing ######
start_time = 140
end_time = 200

# Change to linspace/add slicing here
U_test = test_data['U_star'][...,start_time:end_time]  # 5000 x 2 x 200
P_test = test_data['p_star'][...,start_time:end_time]  # 5000 x 200
T_test = test_data['t'][start_time:end_time]  # 200 x 1
X_test = test_data['X_star']  # 5000 x 2

print(U_test.shape, P_test.shape, T_test.shape, X_test.shape)

N = X_test.shape[0]
T = T_test.shape[0]

# Null vector to test against f and g, and error func
null_vector = torch.zeros((N, 1))
mse = nn.MSELoss()

u_loss_sum, v_loss_sum, p_loss_sum, f_loss_sum, g_loss_sum = [0]*5

for time in range(T):
    # Tidy up data
    UU = U_test[:, 0, time]  # N x T
    VV = U_test[:, 1, time]  # N x T
    PP = P_test[...,time]  # N x T

    TT = np.tile(T_test[time], (1, N))  # N x T

    x_test = X_test[:, 0:1].flatten()[:, None]  # NT x 1
    y_test = X_test[:, 1:2].flatten()[:, None]  # NT x 1
    t_test = TT.flatten()[:, None]  # NT x 1

    u_test = UU.flatten()[:, None]  # N x 1
    v_test = VV.flatten()[:, None]  # N x 1
    p_test = PP.flatten()[:, None]  # N x 1

    # Convert test data to tensors
    x = torch.tensor(x_test, dtype=torch.float32, requires_grad=True)
    y = torch.tensor(y_test, dtype=torch.float32, requires_grad=True)
    t = torch.tensor(t_test, dtype=torch.float32, requires_grad=True)
    u = torch.tensor(u_test, dtype=torch.float32)
    v = torch.tensor(v_test, dtype=torch.float32)
    p = torch.tensor(p_test, dtype=torch.float32)
    
    # Get outputs
    u_prediction, v_prediction, p_prediction, f_prediction, g_prediction = create_pde(net, x, y, t)
    u_loss = mse(u_prediction, u)
    v_loss = mse(v_prediction, v)
    p_loss = mse(p_prediction, p)
    f_loss = mse(f_prediction, null_vector)
    g_loss = mse(g_prediction, null_vector)

    u_loss_sum += u_loss.item()
    v_loss_sum += v_loss.item()
    p_loss_sum += p_loss.item()
    f_loss_sum += f_loss.item()
    g_loss_sum += g_loss.item()

    loss = u_loss + v_loss + p_loss + f_loss + g_loss

    print(f"Timestep {time} loss: {loss.item()}")

print(u_loss_sum/T, v_loss_sum/T, p_loss_sum/T)
print(f_loss_sum/T, g_loss_sum/T)

(14449, 2, 60) (14449, 60) (60, 1) (14449, 2)
Timestep 0 loss: 2621784576.0
Timestep 1 loss: 83512991744.0
Timestep 2 loss: 50383314944.0
Timestep 3 loss: 53539479552.0
Timestep 4 loss: 1902124990464.0
Timestep 5 loss: 2011882586112.0
Timestep 6 loss: 100210745344.0
Timestep 7 loss: 4886119841792.0
Timestep 8 loss: 153933742080.0
Timestep 9 loss: 2884036460544.0
Timestep 10 loss: 1302356033536.0
Timestep 11 loss: 1031577600.0
Timestep 12 loss: 199836008448.0
Timestep 13 loss: 62465900544.0
Timestep 14 loss: 330449289216.0
Timestep 15 loss: 291402350592.0
Timestep 16 loss: 1116305024.0
Timestep 17 loss: 25024942080.0
Timestep 18 loss: 44019490816.0
Timestep 19 loss: 1413400494080.0
Timestep 20 loss: 194692972544.0
Timestep 21 loss: 3653365661696.0
Timestep 22 loss: 1756579758080.0
Timestep 23 loss: 829999546368.0
Timestep 24 loss: 6363115520.0
Timestep 25 loss: 2123792384000.0
Timestep 26 loss: 77554384896.0
Timestep 27 loss: 497101307904.0
Timestep 28 loss: 133022547968.0
Timestep 29 l